In [1]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../MNIST_data/", one_hot=True)

W0225 15:56:28.832053 4650694080 deprecation.py:323] From <ipython-input-1-8bf8ae5a5303>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0225 15:56:28.833268 4650694080 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0225 15:56:28.834394 4650694080 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will 

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz


W0225 15:56:29.061546 4650694080 deprecation.py:323] From /Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import os

from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Activation, Reshape, Embedding, Concatenate
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, Flatten
from keras.layers import LeakyReLU, Dropout
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, plot_model
import keras.backend as K
from keras.datasets import mnist

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image

import numpy as np
np.random.seed(1)

import random
import math

Using TensorFlow backend.


In [3]:
(X_train, _), (_, _) = mnist.load_data()

X_train = X_train.reshape(-1, 28*28)/255.

In [4]:
x_dim = X_train.shape[1] # 784
z_dim = 62
c_dim = 10
h_dim = 128
batch_size = 128

x = Input(shape=(x_dim,))
D_h = Dense(h_dim)(x)
D_h = LeakyReLU(0.2)(D_h)
D_o = Dense(1, activation='sigmoid')(D_h)
DM = Model(x, D_o)

Q_h = Dense(h_dim)(D_h)
Q_h = LeakyReLU(0.2)(Q_h)
Q_o = Dense(c_dim, activation='softmax')(Q_h)
QM = Model(x, Q_o)

z = Input(shape=(z_dim,))
c = Input(shape=(c_dim,))
G_h = Concatenate(axis=1)([z, c])
G_h = Dense(h_dim*2)(G_h)
G_h = LeakyReLU(0.2)(G_h)
G_o = Dense(x_dim, activation='sigmoid')(G_h)
GM = Model([z, c], G_o)

# del DM
# DM = load_model('DM.h5')
# del QM
# QM = load_model('QM.h5')
# del GM
# GM = load_model('GM.h5')

def mutual_info_loss(c, c_given_x):
    """The mutual information metric we aim to minimize"""
    eps = 1e-8
    conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
    entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))
    return conditional_entropy + entropy

DM.trainable = True
DM.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# QM.compile(loss=[mutual_info_loss], optimizer=Adam(), metrics=['accuracy'])

g = GM([z, c])
DM.trainable = False
y = DM(g)
c_given_x = QM(g)
AM = Model([z, c], [y, c_given_x])
AM.compile(loss=['binary_crossentropy', mutual_info_loss], optimizer=Adam(), metrics=['accuracy'])

GM.summary()
DM.summary()
AM.summary()
QM.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 62)           0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 72)           0           input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 256)          18688       concatenate_1[0][0]              
__________

/Users/ianfan/anaconda3/envs/spinningup/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [5]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')
    return fig

def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m,n])

def sample_C(m, n):
    return np.random.multinomial(1, n*[0.1], size=m).astype(np.float32)
#     sampled_labels = np.random.randint(0, n, m).reshape(-1, 1)
#     sampled_labels = np_utils.to_categorical(sampled_labels, num_classes=n)
#     return sampled_labels

if not os.path.exists('out/'):
    os.makedirs('out/')

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))
    
i=0
for it in range(1000000):
    r = np.random.randint(0, X_train.shape[0], batch_size)
    x_real = X_train[r]
    
    z_samples = sample_Z(batch_size, z_dim)
    c_samples = sample_Z(batch_size, c_dim)
    x_fake = GM.predict([z_samples, c_samples])
    
    D_loss_real, D_acc_real = DM.train_on_batch(x_real, real)
    D_loss_fake, D_acc_fake = DM.train_on_batch(x_fake, fake)
    
    D_loss = 0.5*(D_loss_real + D_loss_fake)
    D_acc = 0.5*(D_acc_real + D_acc_fake)
    
    z_samples = sample_Z(batch_size, z_dim)
    c_samples = sample_C(batch_size, c_dim)
    
    G_loss, Q_loss, _, G_acc, Q_acc = AM.train_on_batch([z_samples, c_samples], [real, c_samples])
    
#     z_samples = sample_Z(batch_size, z_dim)
#     c_samples = sample_C(batch_size, c_dim)
    
#     Q_loss, Q_acc = QM.train_on_batch([z_samples, c_samples], c_samples)
    
    if it%10000==0:
        DM.save('DM.h5')
        QM.save('QM.h5')
        GM.save('GM.h5')
        
        for j in range(c_dim):
            c_samples = np.ones((16,1))*j
            c_samples = np_utils.to_categorical(c_samples, num_classes=c_dim)
            samples = GM.predict([sample_Z(16, z_dim), c_samples])
            fig = plot(samples)
            plt.savefig('out/{}.png'.format(str(i).zfill(3) + '_' + str(j)), bbox_inches='tight')
            plt.close(fig)
        i+=1
        
        print('iter:{}'.format(it))
        print('[D loss:{:.4}, acc:{:.4}] [G loss:{:.4}, acc:{:.4}] [Q loss:{:.4}, acc:{:.4}]'.format(D_loss, D_acc, G_loss, G_acc, Q_loss, Q_acc))
        print()

iter:0
[D loss:1.379, acc:0.4023] [G loss:3.007, acc:0.9453] [Q loss:0.6517, acc:0.1094]

iter:10000
[D loss:0.6781, acc:0.6914] [G loss:3.512, acc:0.2031] [Q loss:1.346, acc:0.8828]

iter:20000
[D loss:0.2123, acc:0.9727] [G loss:24.74, acc:0.0] [Q loss:8.049, acc:0.09375]

iter:30000
[D loss:1.096e-07, acc:1.0] [G loss:31.95, acc:0.0] [Q loss:16.12, acc:0.1406]

iter:40000
[D loss:1.096e-07, acc:1.0] [G loss:33.1, acc:0.0] [Q loss:16.12, acc:0.07812]

iter:50000
[D loss:1.096e-07, acc:1.0] [G loss:33.68, acc:0.0] [Q loss:16.12, acc:0.04688]

iter:60000
[D loss:1.096e-07, acc:1.0] [G loss:33.39, acc:0.0] [Q loss:16.12, acc:0.0625]

iter:70000
[D loss:1.096e-07, acc:1.0] [G loss:32.67, acc:0.0] [Q loss:16.12, acc:0.1016]

iter:80000
[D loss:1.096e-07, acc:1.0] [G loss:32.96, acc:0.0] [Q loss:16.12, acc:0.08594]

iter:90000
[D loss:1.096e-07, acc:1.0] [G loss:32.09, acc:0.0] [Q loss:16.12, acc:0.1328]

iter:100000
[D loss:1.096e-07, acc:1.0] [G loss:32.09, acc:0.0] [Q loss:16.12, acc:0.

KeyboardInterrupt: 